# Calibration Rectangle 

150mm x 50mm rectangle for calibration purposes
splay the image.

In [1]:
import gcode
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import requests

esphome_cfg = {
    "host": "cnc.lan",
    "id": "cnc_relay"
}

base_url = f"http://{esphome_cfg['host']}/switch/{esphome_cfg['id']}"
base_url

'http://cnc.lan/switch/cnc_relay'

In [3]:
r = requests.post(f"{base_url}/turn_on")
assert r.status_code == 200

r = requests.get(f"{base_url}")
assert r.json()["state"] == "ON"
assert r.json()["value"]

r = requests.post(f"{base_url}/turn_off")
assert r.status_code == 200


r = requests.get(f"{base_url}")
assert r.json()["state"] == "OFF"
assert not r.json()["value"]

In [4]:
import sys

# https://github.com/jed-frey/build_opencv
sys.path.append("/opt/opencv4/lib/python3.8/site-packages")

import cv2
from PIL import Image
from IPython.display import display

In [5]:
import timeout_decorator

class VideoCapture2(cv2.VideoCapture):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
    @property
    def frame(self):
        ret, frame = self.read()
        if ret:
            return frame
        else:
            raise Exception("Reading Failed")
        
    def pil_img(self):
        img = cv2.cvtColor(self.frame, cv2.COLOR_BGR2RGB)
        return Image.fromarray(img)
    
    def __enter__(self):
        return self
    
    @timeout_decorator.timeout(10)
    def __exit__(self,type, value, traceback):
        while self.isOpened():
            self.release() 

            
import IPython.core.magic
@IPython.core.magic.register_line_magic
def webcam(*args, **kwargs):
    webcams = [arg for arg in args[0].split(" ")]
    for webcam in webcams:
        if webcam.startswith("/dev/video"):
            webcam = int(webcam.split("/dev/video")[-1])
        else:
            try:
                webcam = int(webcam)
            except:
                pass
        with VideoCapture2(webcam) as cap:
            img = cap.pil_img()
            display(img)

In [6]:
import os
auth = os.environ["WEBCAM_AUTH"]
ip   = os.environ["WEBCAM_IP"]

In [ ]:
%webcam rtsp://$auth@$ip/cam/realmonitor?channel=1&subtype=0

In [7]:
r = requests.post(f"{base_url}/turn_on")
assert r.status_code == 200

r = requests.get(f"{base_url}")
assert r.json()["state"] == "ON"
assert r.json()["value"]

import grbl

with grbl.Grbl(port="/dev/ttyUSB0") as cnc:
    cnc.home()

In [8]:
def init():
    prog = gcode.GCode()
    prog.G92(X=0, Y=0, Z=0) 
    prog.G21() # Metric
    prog.G90() # Absolute
    return prog


X_pos = 15
laserin_speed=200
Ys = np.array([10, 60])

def calibration_border():
    laserin_power=255
    laserin_speed=100
    X_pos = 15
    Ys = np.array([10, 60])
    axis_prog=gcode.GCode()
    axis_prog.G0(X=0, Y=0, Z=0, F=300)
    axis_prog.M3(S=int(laserin_power))
    axis_prog.G1(X=0, Y=0, Z=0, F=laserin_speed)
    axis_prog.G1(X=0, Y=Ys[1], Z=0, F=laserin_speed)
    axis_prog.G0(X=X_pos+150, Y=0, Z=0)
    axis_prog.G1(X=0, Y=0, Z=0, F=laserin_speed)
    axis_prog.G1(X=X_pos, Y=Ys[0], Z=0, F=laserin_speed)
    axis_prog.M5()
    return axis_prog

doe=gcode.GCode()
for laserin_power in [75, 100, 125, 150, 175, 200, 255]:
    print(f"Power Test: {laserin_power}")
    for x_spacing in [0, 0.1, 0.25, 0.5, 0.75, 1, 1.25, 1.5, 1.75, 2, 2.5]:
        X_pos+=x_spacing
        X_pos=np.round(X_pos, 4)
        #
        doe.G0(X=X_pos, Y=Ys[0])
        #
        doe.M3(S=int(laserin_power))
        doe.G1(X=X_pos, Y=Ys[0], F=laserin_speed)
        doe.G1(X=X_pos,Y=Ys[1], F=laserin_speed)
        doe.M5()
        #
        doe.G0(X=X_pos, Y=Ys[1])
        #
        print(f"\tX:{X_pos}\tY:{Ys[0]}:{Ys[1]}")
        Ys = np.flip(Ys)
    X_pos+=5
    

Power Test: 75
	X:15	Y:10:60
	X:15.1	Y:60:10
	X:15.35	Y:10:60
	X:15.85	Y:60:10
	X:16.6	Y:10:60
	X:17.6	Y:60:10
	X:18.85	Y:10:60
	X:20.35	Y:60:10
	X:22.1	Y:10:60
	X:24.1	Y:60:10
	X:26.6	Y:10:60
Power Test: 100
	X:31.6	Y:60:10
	X:31.7	Y:10:60
	X:31.95	Y:60:10
	X:32.45	Y:10:60
	X:33.2	Y:60:10
	X:34.2	Y:10:60
	X:35.45	Y:60:10
	X:36.95	Y:10:60
	X:38.7	Y:60:10
	X:40.7	Y:10:60
	X:43.2	Y:60:10
Power Test: 125
	X:48.2	Y:10:60
	X:48.3	Y:60:10
	X:48.55	Y:10:60
	X:49.05	Y:60:10
	X:49.8	Y:10:60
	X:50.8	Y:60:10
	X:52.05	Y:10:60
	X:53.55	Y:60:10
	X:55.3	Y:10:60
	X:57.3	Y:60:10
	X:59.8	Y:10:60
Power Test: 150
	X:64.8	Y:60:10
	X:64.9	Y:10:60
	X:65.15	Y:60:10
	X:65.65	Y:10:60
	X:66.4	Y:60:10
	X:67.4	Y:10:60
	X:68.65	Y:60:10
	X:70.15	Y:10:60
	X:71.9	Y:60:10
	X:73.9	Y:10:60
	X:76.4	Y:60:10
Power Test: 175
	X:81.4	Y:10:60
	X:81.5	Y:60:10
	X:81.75	Y:10:60
	X:82.25	Y:60:10
	X:83.0	Y:10:60
	X:84.0	Y:60:10
	X:85.25	Y:10:60
	X:86.75	Y:60:10
	X:88.5	Y:10:60
	X:90.5	Y:60:10
	X:93.0	Y:10:60
Power Test: 200
	X:98.0

In [9]:
with grbl.Grbl(port="/dev/ttyUSB0") as cnc:
    cnc.reset()
    cnc.home()
    cnc.run(init())
    cnc.run(calibration_border())
    cnc.run(doe)
    cnc.home()

100%|██████████| 462/462 [21:28<00:00,  2.79s/it]


In [ ]:
doe